In [4]:
import chromadb

# Configure ChromaDB with local storage path
client = chromadb.PersistentClient(path="./chroma_db")  # Specify the directory for database files

# Create or access a collection
collection = client.get_or_create_collection("docs_collection")

# Dummy data for testing
collection.add(
    documents=["This is a test document"],
    metadatas=[{"source": "test"}],
    ids=["doc1"]
)

# Example: Query the collection
results = collection.query(query_texts=["test"], n_results=1)

print(results)


C:\Users\SHRIRAM\.cache\chroma\onnx_models\all-MiniLM-L6-v2\onnx.tar.gz: 100%|██████████| 79.3M/79.3M [10:45<00:00, 129kiB/s] 


{'ids': [['doc1']], 'embeddings': None, 'documents': [['This is a test document']], 'uris': None, 'data': None, 'metadatas': [[{'source': 'test'}]], 'distances': [[0.8545427842875148]], 'included': [<IncludeEnum.distances: 'distances'>, <IncludeEnum.documents: 'documents'>, <IncludeEnum.metadatas: 'metadatas'>]}


In [2]:
from PyPDF2 import PdfReader
from langchain.text_splitter import RecursiveCharacterTextSplitter
import os
from langchain_google_genai import GoogleGenerativeAIEmbeddings
import google.generativeai as genai
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.chains.question_answering import load_qa_chain
from langchain.prompts import PromptTemplate
from dotenv import load_dotenv
import chromadb
import hashlib


d:\AI-Companion\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# Configure ChromaDB with local storage path
client = chromadb.PersistentClient(path="./chroma_db")  # Specify the directory for database files
# Create or access a collection
client.delete_collection("docs_collection")
collection = client.get_or_create_collection("docs_collection")


In [ ]:
# load emvironment variables
load_dotenv()
api_key=os.getenv("API")
# one thing i noticed if you device varables have same names as the .env files it will get the device variables
#print(api_key)
genai.configure(api_key="API")

In [9]:
def get_text_from_pdf(file_path):
    pdf = PdfReader(file_path)
    text = ""
    for page in pdf.pages:
        text += page.extract_text()
    return text

In [10]:
def get_text_from_txt(file_path):
    """
    Reads text from a TXT file.
    """
    with open(file_path, "r", encoding="utf-8") as file:
        return file.read()

In [11]:
def get_chunks(text):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=2000, chunk_overlap=300)
    chunks = text_splitter.split_text(text)
    return chunks

In [12]:
print(len(get_chunks("This is a test document"*100)))

2


In [13]:
import time
import random
import hashlib

def generate_doc_id():
    """
    Generates a unique, compact ID  to store vectors in the database.
    rtype: str
    """
    timestamp = int(time.time()).to_bytes(4, 'big')  # 4 bytes of UNIX timestamp
    random_part = random.getrandbits(48).to_bytes(6, 'big')  # 6 bytes of randomness
    unique_bytes = timestamp + random_part  # 10 bytes total 
    
    return hashlib.md5(unique_bytes).hexdigest()  # Full-length MD5 hash (32 chars)

# Example usage
print(generate_doc_id())  # Example output: 'e99a18c428cb38d5f260853678922e03'


ee305913fdde41c60c42fffd72e3c5b8


In [52]:
def store_vectors(chunks):
    """
    IMPORTANT:
    The ChromaDB collection's dimensionality is fixed at the time of creation, based on the first inserted embeddings.
    For example, if the collection was initially populated with 384-dimensional embeddings, it will always expect 384 dimensions.
    If you change the embedding model or modify its configuration (such as using a model that outputs 768-dimensional vectors),
    this will cause a dimensionality mismatch error (e.g., "Embedding dimension 768 does not match collection dimensionality 384").

    In future:
        - DO NOT change the embedding model or its output dimensionality once the collection has been created.
        - If a change is necessary, you must either delete the previous data (or create a new collection) to match the new dimensionality,
        which may not be feasible in a production environment.
        
    To retain full information and avoid having to delete existing data, ensure that the embedding model and its configuration remain consistent.
    """

    # Create an instance of the embeddings class using the correct model
    embeddings_obj = GoogleGenerativeAIEmbeddings(model="models/embedding-001")
    # Generate embeddings for the list of texts using embed_documents
    vectors = embeddings_obj.embed_documents(chunks)
    
    # Iterate through the texts and store each one with its corresponding vector
    for i, text in enumerate(chunks):
        doc_id = generate_doc_id()  # Generate a unique ID for this text
        collection.add(
            ids=[doc_id],
            embeddings=[vectors[i]],  # vector for this text
            documents=[text]            # original text (optional)
        )

def test_embedding_storage():
    # Define dummy texts
    txt="""Black Holes: The Mysterious Cosmic Phenomena
Black holes are regions in space where the gravitational pull is so strong that nothing, not even light, can escape. They are formed when massive stars collapse under their own gravity, leading to the creation of an incredibly dense singularity at the center. The boundary surrounding this singularity is called the event horizon, beyond which nothing can return.

There are three primary types of black holes:

Stellar Black Holes – These form from the remnants of massive stars after they undergo supernova explosions. They typically have masses ranging from a few to tens of times that of the Sun.
Supermassive Black Holes – Found at the center of most galaxies, these black holes have masses millions or even billions of times that of the Sun. The supermassive black hole at the center of the Milky Way is called Sagittarius A and has a mass of about 4.3 million suns.
Intermediate Black Holes – These black holes are believed to have masses between stellar and supermassive black holes, but their existence is still under research.
Black holes can only be detected indirectly, as they do not emit light. Scientists observe the effects they have on nearby objects, such as stars orbiting them at extreme speeds, gravitational lensing, and the emission of X-rays from material falling into them. The first direct image of a black hole was captured in 2019 by the Event Horizon Telescope (EHT), showing the supermassive black hole at the center of the M87 galaxy.

The concept of black holes has led to numerous theories in physics, including those related to time travel, wormholes, and Hawking radiation—a process theorized by Stephen Hawking where black holes slowly lose mass due to quantum mechanical effects."""
    dummy_texts = [
        txt
    ]
    
    # Store embeddings in the collection
    store_vectors(dummy_texts)
    
    # Retrieve stored data from the collection
    stored_data = collection.get()
    
    # Print stored IDs and documents for inspection
    print("Stored IDs:", stored_data["ids"])
    print("Stored Documents:", stored_data["documents"])
    print("Total stored entries:", len(stored_data["ids"]))
    
    # Check that all dummy texts were stored
    assert len(stored_data["ids"]) >= len(dummy_texts), "Error: Some embeddings are missing!"
    print("Test Passed: All embeddings stored successfully!")
# Run the test function
test_embedding_storage()

Stored IDs: ['bcc47f9ac6f08a8c87bf036ca27a993d']
Stored Documents: ['Black Holes: The Mysterious Cosmic Phenomena\nBlack holes are regions in space where the gravitational pull is so strong that nothing, not even light, can escape. They are formed when massive stars collapse under their own gravity, leading to the creation of an incredibly dense singularity at the center. The boundary surrounding this singularity is called the event horizon, beyond which nothing can return.\n\nThere are three primary types of black holes:\n\nStellar Black Holes – These form from the remnants of massive stars after they undergo supernova explosions. They typically have masses ranging from a few to tens of times that of the Sun.\nSupermassive Black Holes – Found at the center of most galaxies, these black holes have masses millions or even billions of times that of the Sun. The supermassive black hole at the center of the Milky Way is called Sagittarius A and has a mass of about 4.3 million suns.\nInterm

In [15]:
def process_files(files):
    # Yet to be tested
    for file in files:
        if file.endswith(".pdf"):
            text=get_text_from_pdf(file)
        elif file.endswith(".txt"):
            text=get_text_from_txt(file)
        else:
            print("File type not supported")
            continue
        chunks=get_chunks(text)# get chunks
        store_vectors(chunks)# store vectors

In [70]:


# Ensure the model used here is the SAME as the one used during document insertion
embeddings_model = GoogleGenerativeAIEmbeddings(model="models/embedding-001")

query_vector = embeddings_model.embed_query("What are black holes?")

results = collection.query(
    query_embeddings=[query_vector],  # Now using correctly-sized embeddings
    n_results=3  
)

print("Retrieved Documents:", results["documents"])
print("Distances:", results["distances"])


Number of requested results 3 is greater than number of elements in index 1, updating n_results = 1


Retrieved Documents: [['Black Holes: The Mysterious Cosmic Phenomena\nBlack holes are regions in space where the gravitational pull is so strong that nothing, not even light, can escape. They are formed when massive stars collapse under their own gravity, leading to the creation of an incredibly dense singularity at the center. The boundary surrounding this singularity is called the event horizon, beyond which nothing can return.\n\nThere are three primary types of black holes:\n\nStellar Black Holes – These form from the remnants of massive stars after they undergo supernova explosions. They typically have masses ranging from a few to tens of times that of the Sun.\nSupermassive Black Holes – Found at the center of most galaxies, these black holes have masses millions or even billions of times that of the Sun. The supermassive black hole at the center of the Milky Way is called Sagittarius A and has a mass of about 4.3 million suns.\nIntermediate Black Holes – These black holes are be

In [40]:
try:
    models = genai.list_models()  # List available models
    print("API Key is valid. Available models:", [model.name for model in models])
except Exception as e:
    print("Error: Invalid API Key or API issue:", e)

API Key is valid. Available models: ['models/chat-bison-001', 'models/text-bison-001', 'models/embedding-gecko-001', 'models/gemini-1.0-pro-latest', 'models/gemini-1.0-pro', 'models/gemini-pro', 'models/gemini-1.0-pro-001', 'models/gemini-1.0-pro-vision-latest', 'models/gemini-pro-vision', 'models/gemini-1.5-pro-latest', 'models/gemini-1.5-pro-001', 'models/gemini-1.5-pro-002', 'models/gemini-1.5-pro', 'models/gemini-1.5-flash-latest', 'models/gemini-1.5-flash-001', 'models/gemini-1.5-flash-001-tuning', 'models/gemini-1.5-flash', 'models/gemini-1.5-flash-002', 'models/gemini-1.5-flash-8b', 'models/gemini-1.5-flash-8b-001', 'models/gemini-1.5-flash-8b-latest', 'models/gemini-1.5-flash-8b-exp-0827', 'models/gemini-1.5-flash-8b-exp-0924', 'models/gemini-2.0-flash-exp', 'models/gemini-2.0-flash', 'models/gemini-2.0-flash-001', 'models/gemini-2.0-flash-lite-preview', 'models/gemini-2.0-flash-lite-preview-02-05', 'models/gemini-2.0-pro-exp', 'models/gemini-2.0-pro-exp-02-05', 'models/gemini-

In [69]:
import getpass
import os

if not os.getenv("GOOGLE_API_KEY"):
    os.environ["GOOGLE_API_KEY"] = getpass.getpass("Enter your Google AI API key: ")
print("DOne")
#print(os.environ["GOOGLE_API_KEY"])

DOne


In [ ]:
def get_conversational_chain():
    """Loads the conversational chain with Gemini API"""
    prompt_template = """
   You are a knowledgeable assistant. Use the provided context to answer the question in detail. If the answer is not in the context, say: 'The answer is not available in the provided context,' then continue answering based on what you know.'
    
    Context:\n{context}\n
    Question:\n{question}\n
    
    Answer:
    """
    model = ChatGoogleGenerativeAI(model="gemini-pro",api_key="KEY",temperature=0.3)
    prompt = PromptTemplate(template=prompt_template, input_variables=["context", "question"])
    chain = load_qa_chain(model, chain_type="stuff", prompt=prompt)

    return chain

from langchain.schema import Document  # Import Document

def ask_llm(user_prompt):
    """Queries the LLM after retrieving relevant documents"""
    
    # Initialize embeddings
    embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")  # Ensure this model exists

    # Initialize ChromaDB client and collection
    client = chromadb.PersistentClient(path="./chroma_db")
    collection = client.get_or_create_collection(name="docs_collection")

    # Perform similarity search
    results = collection.query(
        query_embeddings=[embeddings.embed_query(user_prompt)],
        n_results=1  
    )

    print(results)

    # Load conversational chain
    chain = get_conversational_chain()
    
    # Retrieve documents and convert them to Document objects
    docs = [
        Document(page_content=doc) for doc in results["documents"][0]
    ] if results["documents"] else []

    # Generate response
    response = chain(
        {"input_documents": docs, "question": user_prompt},
        return_only_outputs=True
    )

    return response


print(ask_llm("What are black holes"))


{'ids': [['bcc47f9ac6f08a8c87bf036ca27a993d']], 'embeddings': None, 'documents': [['Black Holes: The Mysterious Cosmic Phenomena\nBlack holes are regions in space where the gravitational pull is so strong that nothing, not even light, can escape. They are formed when massive stars collapse under their own gravity, leading to the creation of an incredibly dense singularity at the center. The boundary surrounding this singularity is called the event horizon, beyond which nothing can return.\n\nThere are three primary types of black holes:\n\nStellar Black Holes – These form from the remnants of massive stars after they undergo supernova explosions. They typically have masses ranging from a few to tens of times that of the Sun.\nSupermassive Black Holes – Found at the center of most galaxies, these black holes have masses millions or even billions of times that of the Sun. The supermassive black hole at the center of the Milky Way is called Sagittarius A and has a mass of about 4.3 millio

In [58]:
stored_embeddings = collection.get(include=["embeddings"])
print(len(stored_embeddings["embeddings"][0]))  # Should print 768


768
